# This notebook will run the FRD setup
* To perform an FRD test (before running this script), you should:
1. Check serial connection to 1. CMOS camera 2. Linear stage 3. Iris
2. Make sure to put the cover on the camera
3. Clean both fiber input and output surface
4. On the fiber input end, check with the SpinView software, to make sure the fiber is 1. in focus and 2. clean, also make sure the input beam is injected into the core of the fiber
5. Make sure you change the fiber serial number and the date (if needed) in the '3. Make Directory' shell
6. Run this script

In [ ]:
# Imports

# 1. Camera
import os
import sys
import zwoasi as asi

from IPython.display import clear_output, Image, display
import ipywidgets
import cv2
import time
import numpy as np
import warnings
from astropy.modeling.models import Gaussian2D, AiryDisk2D
from astropy.modeling import fitting
from astropy.nddata.utils import Cutout2D
from astropy.io import fits
from astropy.time import Time
from astropy.visualization import simple_norm
from astropy.nddata import block_reduce

from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus, ApertureStats
from photutils.centroids import centroid_quadratic, centroid_com, centroid_1dg, centroid_2dg
from photutils.profiles import RadialProfile
from PIL import Image, ImageFont, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from pathlib import Path
import cv2
import ipywidgets


# 2. Stage
from libNewport import smc100

# 3. IRIS
from libStanda import standa


# 1. Serial port connection

In [4]:
# 3. IRIS
iris = standa('COM4')

# 2. Stage
s=smc100('COM6')

# Check stage status, if status=ready, no need do homing
if s.state[1]['CURSTATE'] != 'READY':
    print(f"Controller Status: {s.state[1]['CURSTATE']}- Controller not ready. Reset & Home now.")
    s.resetController()
    s.homeController()
    print("")
    print("Controller ready now ---")
    #print("Let's GOOOOOOO")
    #print("=============================================")
else:
    print(f"Controller Status: {s.state[1]['CURSTATE']}")
    print("")
    print("Controller ready now ---")
    #print("Let's GOOOOOOO")
    #print("=============================================")


Trying to autoconnect to serial on COM4
Found 1 real device(s):
  {'uri': 'xi-com:\\\\.\\COM4', 'device_serial': 36292, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 3, 'Release': 6, 'ControllerName': '8MID27-1.5-AR', 'CtrlFlags': 0, 'PositionerName': ''}
Success. Opened controllers on port COM4
Trying to autoconnect to serial on COM6
Success. Opened controllers on port COM6
Stage 1IDCMA-25CCCL_PN:091207_UD:090319 on channel 1 has state READY and position 1TP11.99997
Controller Status: READY

Controller ready now ---


# 2. Camera set up
- If connection fails, run the shell for one more time. It usually takes 2 times.

In [5]:
# 1. Check camera .dll file 
ASI_filename = 'lib\\ASICamera2.dll'

if ASI_filename:
    asi.init(ASI_filename)
else:
    print('The filename of the ASI SDK library is required')
    sys.exit(1)
    
# 2. Check ZWO ASI1600MM Pro camera connection
num_cameras = asi.get_num_cameras()
if num_cameras == 0:
    print('No cameras found')
    sys.exit(0)
else:
    cameras_found = asi.list_cameras()  # Models names of the connected cameras
    print("Camera found:", cameras_found)

Camera found: ['ZWO ASI1600MM Pro']


In [6]:
# 3. CONNECT to CAMERA
# open camera handle
#camera_id = 0  # use first camera from list
#camera_id = index[0]  # use first camera from list
camera_id = 0  # 
camera = asi.Camera(camera_id)
camera_info = camera.get_camera_property()

In [7]:
# 4. Camera info
if (False):
# print(camera_info)
    for key,value in camera_info.items():
        print(f"{key} :  {value}" )

pixelsize = camera_info['PixelSize']
maxheight =  camera_info['MaxHeight']
maxwidth =  camera_info['MaxWidth']

if (False):
# Get all of the camera controls
    print('')
    print('Camera controls:')
    controls = camera.get_controls()
    for cn in sorted(controls.keys()):
        print('    %s:' % cn)
        for k in sorted(controls[cn].keys()):
            print('        %s: %s' % (k, repr(controls[cn][k])))

In [8]:
# 5. Get all of the current camera control values
#print('')
#print('Camera control values: [value, auto(bool)]')
set_exptime = 0.03 # seconds
set_exptime = int(set_exptime*10**6) # convert to microseconds
camera.set_control_value(asi.ASI_EXPOSURE, set_exptime)  # microseconds

#camera.set_control_value(asi.ASI_AUTO_MAX_BRIGHTNESS, 30000)

# 6. confirm exptime was set
get_exptime = camera.get_control_value(asi.ASI_EXPOSURE)
current_temp = camera.get_control_value(asi.ASI_TEMPERATURE)  # returns x10 for precision 
current_temp[0] = current_temp[0]/10
exptime = get_exptime[0]/10**6
temperature = current_temp[0]
print('Exposure time=', exptime)
print('Temperature=',temperature)  

#fan_on = camera.get_control_value(asi.ASI_FAN_ON)
#cooler_on = camera.get_control_value(asi.ASI_COOLER_ON)
#cooler_power = camera.get_control_value(asi.ASI_COOLER_POWER_PERC)
#target_temp = camera.get_control_value(asi.ASI_TARGET_TEMP)

#print(fan_on, cooler_on, cooler_power, target_temp, current_temp)
#print(current_temp)

if (False):
    # Use minimum USB bandwidth permitted
    camera.set_control_value(asi.ASI_BANDWIDTHOVERLOAD, camera.get_controls()['BandWidth']['MinValue'])
    
    # Set some sensible defaults. They will need adjusting depending upon
    # the sensitivity, lens and lighting conditions used.
    camera.disable_dark_subtract()
    
    #camera.set_control_value(asi.ASI_GAIN, 150)
    #camera.set_control_value(asi.ASI_EXPOSURE, 1300000) # microseconds


Exposure time= 0.03
Temperature= 28.0


# 3. Make Directory

In [40]:
from datetime import datetime

# Get today's date
today = datetime.today()
# Format it as a string
formatted_date = today.strftime('%Y-%m-%d')
print("Today's date=",formatted_date)

# Define the directory path dynamically
dirpath = Path(f'results/{formatted_date}-lfast-18um-0032')
print("Output directory path:", dirpath)

filename = f"image_{formatted_date}_target001.fits"
fpath = dirpath / filename


mkdir = True
if mkdir:
    dirpath_mkdir=Path(dirpath)
    dirpath_mkdir.mkdir(parents=True, exist_ok=True)

Today's date= 2024-10-04
dirpath= C:/Users/CFBLAB1/Desktop/LFAST-FRD/2024-10-04-lfast-18um-0032


# 4. Set f ratio of the input beam

In [54]:
# define iris apertures [mm] for input f/ratio's  determined by focal length of objective (40mm) / aperture 
# iris apertures range from 1.5-27mm, verify that the collimated beam can actually illuminate the desired range.  

focal_length = 20   # Nikon 10x plan Fluor microscope objective NA = 0.3  minimum useful fratio = 1/2NA = f/1.666.  
fratios = np.array([1.7, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8.0])

apertures = focal_length / fratios

# or define fixed apertures
#apertures = [23.53, 20.0, 16, 13.33, 11.43, 10.0, 8.89, 8.0, 6.67, 5.71, 5.0, 4.44, 4.0]

# define exposure times in milliseconds to expose for each aperture setting. 
# 2000ms for 20mm aperture, scale by area.  Note, not quite true since the area illiminated will change.
# If there was no FRD then the area will increase as the sqare and the result would cancel... ie same exposure time. 
# since there is FRD larger apertures will be close in exptime and smaller apertures will need to be exposed longer to get same S/N.
#texp = 500 
texp = 2000

# update stage position and exptime based on input f ratio?
dynamic = False

#exptimes = texp * (20)**2 / (apertures)**2
if dynamic:
    #exptimes = np.floor(texp*( (1+np.exp(-apertures/10)/np.exp(-1))/2 )/100)*100
    exptimes = texp*(1+0*apertures)
else:
    exptimes = texp*(1+0*apertures)
print(exptimes)
print(apertures) 

[2000. 2000. 2000. 2000. 2000. 2000. 2000. 2000. 2000. 2000. 2000. 2000.]
[11.76470588 10.          8.          6.66666667  5.71428571  5.
  4.44444444  4.          3.63636364  3.33333333  2.85714286  2.5       ]


# 5. Start loop

In [55]:
# start loop:
#1. set filter
#2. move stage
#3. take pic & save


for i,aperture in enumerate(apertures): # cycle through apertures

    print(i, aperture, fratios[i])
    # minimum spot position is at ~85mm and the BFD of the ZWOASI1600 is 6.5mm,
    # so the closest the stage can be is 85-6.5=78.5mm before the fiber touches the window

    #adjust stage distance for each input f ratio?
    if dynamic :
        if fratios[i] >= 0: positions = np.arange(2,16,3) #0mm is closet to fiber, 25mm is farthest
        if fratios[i] >= 4: positions = np.arange(2,23,5)
        if fratios[i] >= 8: positions = np.arange(2,25,4)
        if fratios[i] >= 12: positions = np.arange(2,25,4)
    else:
        positions = np.arange(0,13,4)
        
    
    #for pos in range(65, 44, -5): # Iterate over absolute positions 65 to 45
    #for pos in range(76, 63, -3): # Iterate over absolute positions 76 to 63  
    print(positions)
    #continue
    
    iris.setPositionAbs(aperture) # change filter  
    print("IRIS Aperture:  ", i, aperture)
    
    for j,pos in enumerate(positions): # Iterate over absolute positions

        # Reset exp time
        if dynamic :
            set_exptime = exptimes[i]/1000 # seconds
        else:
            set_exptime = exptimes[i]/1000 # seconds
            
        set_exptime = int(set_exptime*10**6) # convert to microseconds
        camera.set_control_value(asi.ASI_EXPOSURE, set_exptime)  # microseconds
    
        # Confirm new exptime was set
        get_exptime = camera.get_control_value(asi.ASI_EXPOSURE)
        exptime = get_exptime[0]/10**6
        print('New Exposure time=', exptime)
    
        time.sleep(1)
    
        
        # 1. Move stage ====================================================
        print(f"Go to position at {pos}mm")
        s.setPositionAbs(pos)
        
        # Query current position and wait for 5 seconds
        current_pos = s.getPosition()
        print(f"Current position (final): {current_pos[3:12]}mm")
        time.sleep(1) # waiting time for priniting position query >_<
        current_pos = s.getPosition()
        print(f"Current position (final): {current_pos[3:12]}mm")        
        print(f"Now at {pos}mm.")
        
        
        # 2. Take pictures =================================================
        
        # 2.1 Set file name
        base_filename = "Aper-{filter:02d}-d{d:02d}"
        this_filename= base_filename.format(filter=i, d=pos)
        print()
        print("Test = ",this_filename)
        
        # 2.2 Take mono16 image
        ####### add some waiting time for camera 
        print("Now capture image...")
        time.sleep(1) # wait time before take pictures
        camera.set_image_type(asi.ASI_IMG_RAW16) # it means mono16
        
        #if filename == None:
        img = camera.capture()
        time.sleep(1) #wait time after taking pictures
        
        # 2.3 Display the image
        #print(img)
        plt.figure(figsize=(4,3))
        plt.imshow(img, cmap='gray')
        plt.colorbar()
        plt.show()

        # 2.4 save image
        print("Saving image...")
        t=Time.now()
        dateobs = t.isot
        filename = this_filename +"_"+t.strftime('%Y%m%d_%H%M%S')
        
        hdu = fits.PrimaryHDU(image_data) ##
        hdu.writeto(fpath, overwrite=True) ##
      
        print("Image saved at: ", fpath)

        # create fits image for raw data
        filename = fpath.as_posix()
        hdu = fits.PrimaryHDU(data=img)
        hdu.writeto(filename)
        fits.setval(filename,'date-obs',value=t.fits)
        fits.setval(filename,'exptime',value=exptime)
        fits.setval(filename,'ccd-temp',value=temperature)
        fits.setval(filename,'irisaper',value=aperture)
        fits.setval(filename,'distance',value=pos)
        print("==========================================================================================")

0 11.764705882352942 1.7
[ 0  4  8 12]
moving...
arrived at:11.765
IRIS Aperture:   0 11.764705882352942
New Exposure time= 2.0
Go to position at 0mm
ERROR: 1TP3.93720
Current position: 1TS000028 ERROR
ERROR: 1TP3.37607
Current position (final): 000028mm
ERROR: 1TP2.81449
Current position (final): 000028mm
Now at 0mm.

Test =  Aper-00-d00
Now capture image...


ZWO_CaptureError: Could not capture image

# 6. Close Serial Connection

In [56]:
# Close the serial connection

# camera
camera.close()

# Stage
print("Finish taking all images. YAY. Close seial connection now.")
s.ser.close()
print("Stage connection closed")

# iris
iris.close()
print("Iris connection closed")

Finish taking all images. YAY. Close seial connection now.
Stage connection closed
Iris connection closed
